In [52]:
!pip install gensim
!pip install xgboost
!pip install catboost
!pip install lightgbm 
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


#### Import Libraries

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import matplotlib.style as style # for styling the graphs
# style.available (to know the available list of styles)
style.use('ggplot') # chosen style
plt.rc('xtick',labelsize=13) # to globally set the tick size
plt.rc('ytick',labelsize=13) # to globally set the tick size
# To print multiple outputs together
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
# Change column display number during print
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

#### Load Dataset

In [20]:
path_1 = './Feature-Engineering-Train.csv'
path_2 = './Feature-Engineering-Test.csv'

In [21]:
# Created Features based on Lead and lag timeseries features , Yoy Growth Feature, Created Fatures related to Season, Aggregate Features
train_df = pd.read_csv(path_1)
val_df = pd.read_csv(path_2)

#### Drop sku name column

In [22]:
train_df.drop('sku_name',1,inplace=True)
val_df.drop('sku_name',1,inplace=True)

#### Fill NaN values as 0 in ratio inventory column

In [23]:
train_df['ratio_inventory'].fillna(0,inplace=True)
val_df['ratio_inventory'].fillna(0,inplace=True)

#### Drop negative sellin values in target column

In [24]:
rows_negative_vlaues = val_df[val_df["sellin"]<0]
val_df.drop(rows_negative_vlaues.index, axis=0, inplace = True)

rows_negative_vlaues = train_df[train_df["sellin"]<0]
train_df.drop(rows_negative_vlaues.index, axis=0, inplace = True)

In [25]:
# Replace 'inf' values in the 'YoY' feature with a reasonable value, such as the mean of the feature
train_df['YoY_growth'] = np.where(np.isinf(train_df['YoY_growth']),0, train_df['YoY_growth'])
val_df['YoY_growth'] = np.where(np.isinf(val_df['YoY_growth']),0, val_df['YoY_growth'])

#### Create a ML Model

In [27]:
import pandas as pd  # Import pandas for DataFrame handling
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
from tqdm import tqdm
import numpy as np
from sklearn.linear_model import LinearRegression

# Training features
X = np.array(train_df.drop('sellin', axis=1))
y = np.array(np.log1p(train_df["sellin"]))

# Validation features
X_test = np.array(val_df.drop('sellin', axis=1))
y_test = np.array(np.log1p(val_df["sellin"]))

# Define the models
models = [
    ("Decision Tree", DecisionTreeRegressor(random_state=9)),
     ("Random Forest", RandomForestRegressor(n_estimators=100, random_state=42)),
     ("XGBoost", XGBRegressor(n_estimators = 100,objective='reg:squarederror', random_state=42)),
   ("CatBoost", CatBoostRegressor(n_estimators = 100,verbose=0, random_state=42)),
     ("LightGBM", LGBMRegressor(n_estimators = 100,random_state=42))
]

# Define the KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# DataFrame to store feature importances
feature_importances_df = pd.DataFrame(index=train_df.drop(['sellin'],1).columns)

# Train and evaluate each model
for name, model in tqdm(models, desc="Evaluating Models"):
    rmse_scores = []  # To store RMSE scores for each fold
    mae_scores = []  # To store MAE scores for each fold
    mape_scores = []  # To store MAPE scores for each fold
    evs_scores = []  # To store Explained Variance Scores for each fold
    fold_importances = []  # To store feature importances for each fold
    
    # K-fold cross-validation
    for train_index, val_index in kf.split(X):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        # Train the model on the fold
        model.fit(X_train_fold, y_train_fold)
        
        # Predict and reverse the log transformation on validation set
        y_pred = np.expm1(model.predict(X_val_fold))
        y_val_original = np.expm1(y_val_fold)
        
        # Calculate metrics
        rmse_scores.append(np.sqrt(mean_squared_error(y_val_original, y_pred)))
        mae_scores.append(mean_absolute_error(y_val_original, y_pred))
        mape_scores.append(mean_absolute_percentage_error(y_val_original, y_pred))
        evs_scores.append(explained_variance_score(y_val_original, y_pred))

        # Store feature importances if supported
        if hasattr(model, "feature_importances_"):
            fold_importances.append(model.feature_importances_)

    # Store average feature importances for the model
    if fold_importances:
        avg_importances = np.mean(fold_importances, axis=0)
        feature_importances_df[name] = avg_importances
    
    # Print metrics
    print(f"{name} Average RMSE: {np.mean(rmse_scores):.4f}")
    print(f"{name} Average MAE: {np.mean(mae_scores):.4f}")
    print(f"{name} Average MAPE: {np.mean(mape_scores):.4f}")
    print(f"{name} Average Explained Variance Score: {np.mean(evs_scores):.4f}")

    # Final evaluation on the test set
    y_pred_test = np.expm1(model.predict(X_test))
    y_test_original = np.expm1(y_test)

    final_rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_test))
    final_mae = mean_absolute_error(y_test_original, y_pred_test)
    final_mape = mean_absolute_percentage_error(y_test_original, y_pred_test)
    final_evs = explained_variance_score(y_test_original, y_pred_test)
    
    print(f"{name} Test RMSE: {final_rmse:.4f}")
    print(f"{name} Test MAE: {final_mae:.4f}")
    print(f"{name} Test MAPE: {final_mape:.4f}")
    print(f"{name} Test Explained Variance Score: {final_evs:.4f}")

# Display the feature importances
print("\nFeature Importances:")
print(feature_importances_df)


Evaluating Models:   0%|                                                                         | 0/5 [00:00<?, ?it/s]

DecisionTreeRegressor(random_state=9)

DecisionTreeRegressor(random_state=9)

DecisionTreeRegressor(random_state=9)

DecisionTreeRegressor(random_state=9)

...

LGBMRegressor(random_state=42)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030336 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 21446
[LightGBM] [Info] Number of data points in the train set: 34414, number of used features: 113
[LightGBM] [Info] Start training from score 10.141017


LGBMRegressor(random_state=42)

Evaluating Models: 100%|████████████████████████████████████████████████████████████████| 5/5 [38:01<00:00, 456.23s/it]

LightGBM Average RMSE: 77984.3764
LightGBM Average MAE: 17947.6393
LightGBM Average MAPE: 0.1018
LightGBM Average Explained Variance Score: 0.9784
LightGBM Test RMSE: 80653.6961
LightGBM Test MAE: 17340.4140
LightGBM Test MAPE: 0.0899
LightGBM Test Explained Variance Score: 0.9771

Feature Importances:
                                     Decision Tree  Random Forest   XGBoost  \
starting_inventory                    3.458606e-05   4.181932e-05  0.000122   
sellin_channel_1                      6.795502e-01   6.817416e-01  0.533319   
sellin_channel_2                      5.203112e-03   4.957614e-03  0.012468   
sellin_channel_3                      9.064994e-03   9.630366e-03  0.032026   
sellin_channel_4                      3.584638e-02   3.423365e-02  0.049617   
sellin_channel_5                      2.238541e-03   2.104265e-03  0.008667   
sellin_channel_6                      5.790112e-02   5.375552e-02  0.078911   
sellin_channel_7                      1.135025e-04   1.096200e-0

In [28]:
feature_importances_df.sort_values(["LightGBM"],ascending=False)

,Decision Tree,Random Forest,XGBoost,CatBoost,LightGBM
leftover_inventory,1.639317e-01,1.630299e-01,0.113396,24.819694,468.2
sellin_channel_1,6.795502e-01,6.817416e-01,0.533319,22.302039,351.6
sellout,1.249309e-02,1.994107e-02,0.023661,9.460666,311.2
sellin_channel_4,3.584638e-02,3.423365e-02,0.049617,9.436006,268.6
sellin_channel_6,5.790112e-02,5.375552e-02,0.078911,15.873022,196.4
sellin_channel_8,1.295099e-02,1.286856e-02,0.031894,3.753775,173.4
sellin_channel_2,5.203112e-03,4.957614e-03,0.012468,1.541555,129.6
sellin_channel_3,9.064994e-03,9.630366e-03,0.032026,5.237989,125.2
sellout_channel_10,2.882674e-03,2.776515e-03,0.010633,0.727903,110.2
sellin_channel_5,2.238541e-03,2.104265e-03,0.008667,0.592301,98.4
